# Calculation
A lot of the design work has been done in the [Zombocalyps-project](https://github.com/LieBtrau/zombocalypse-emcomms/blob/main/amplifier/amplifier.ipynb).

## Advice from the Infineon AN_1803 application note
<img src="./images/principle_schematic_class_E.png" width="500px"/>

1. Keep the impedance of L2 and C2 as low as possible to minimize the losses.
2. Q must be > 2.5
3. Check voltage range and current for C2 -> high voltage NP0 or C0G capacitor
4. In correctly tuned operation Vds = 3.56 * Vdd.  Incorrectly tuned operation can lead spikes up to 7 * Vdd.  Protect your transistor against that.
5. Keep Coss low.  Transistor with high Vds breakdown voltage have higher Coss.
6. Choose C1 >> Coss, because Coss is Vds dependent.

# Transistor selection
* FET-type : N-channel
* Vdss > Vdd * 3.56 * 1.4 (safety factor) = 60V
* Id > Id_max * 1.4 (safety factor) = 1.68A -> 2A (Id_max from simulation is 1.2A.)
* Rds_on < 0.3 Ohm at Vgs = 4.5V and Id = 2A
* Vgs < 2.5V
* Qg < 4nC
* Cost < €1
* Availability : JLCPB
* Package : SOT-23 or similar
* Coss @ 10V < 160pF (Coss_max from simulation is << 160pF.)

## Candidates
| Part Number       | Coss [pF] | Vgs [V]     | Qg [nC]    | Rds_on [mOhm]  | Availability          |
|-------------------|-----------|-------------|------------|----------------|-----------------------|
| IRLML0060TRPBF    | 50        | 1.0-2.5     | 2.5        | 116            | JLCPCB : OK           |
| SQ2364EES-T1_GE3  | 40        | 0.46-1.0    | 2-2.5      | 245            | JLCPCB : OK           |
| RQ5L020SNTL       | 50        | 1.0-2.5     | 2.7        | -              | not available at JLCPCB|
| RSR020N06         | 50        | 1.0-2.5     | 2.7        | -              | not available at JLCPCB|

In [1]:
import math
Vgs = 5 # Gate-source voltage [V]

# IRLML0100
Rg = 1.3 # Internal gate resistance [Ohm], add output resistance of the driver here
Ciss = 290e-12 # Input capacitance [F], at Vds=0V (see datasheet, fig.5)
Vth = 2.5 # Threshold voltage [V]
Vgp = 2.8 # Miller plateau voltage [V] (it's the value of the horizontal line in the gate charge curves in the datasheet, fig.6)

# Turn-on delay
td_on = Rg * Ciss * math.log(Vgs/(Vgs-Vgp)) # t2, eq. 12 = t1 + t_ir = eq. 11 + eq. 17
print(f"Turn-on delay: {td_on*1e12:.2f} ps")

# Turn-off delay
td_off = Rg * Ciss * math.log(Vgs/Vgp) # t4, eq. 14
print(f"Turn-off delay: {td_off*1e12:.2f} ps")

# Turn-off delay (alternative method)
td_off = Rg * Ciss * math.log(Vgs/(Vth)) # t4 + t6, eq. 14 + eq. 16
print(f"Turn-off delay (alternative method): {td_off*1e12:.2f} ps")

# Maximum value for C1+Coss (Infineon AN-1803, eq. 20)
f = 13.56e6 # Switching frequency [Hz]
Z_tx_in = 15 # Input impedance of the matching network [Ohm]
C1_Coss = 0.1836 / (2*math.pi*f*Z_tx_in) # [F]
print(f"Maximum value for C1+Coss: {C1_Coss*1e12:.2f} pF")


Turn-on delay: 309.51 ps
Turn-off delay: 218.59 ps
Turn-off delay (alternative method): 261.32 ps
Maximum value for C1+Coss: 143.66 pF




## Inductor criteria
Use smallest inductor value possible to minimize core loss.
* DC resistance
* AC resistance (skin effect, proximity effect)
* core loss (frequency dependent)

# References
* [FesZ Electronics : Class E RF amplifier - Basics (1/3)](https://www.youtube.com/watch?v=Tgrakttus3c)
* [Class-E power amplifier design for wireless power transfer, Infineon AN_1803_PL16_1803_210431](https://www.infineon.com/dgdl/Infineon-Wireless_power_transfer_class_E_power_amp-AN-v01_02-EN.pdf)